In [ ]:
from phobosdb import DBReader

async with DBReader() as db:
    print(await db.async_fetch("select * from min_bars limit 5;"))

In [2]:
# pyre-strict
# Notes
# To run the file, follow the following steps:
# -- 1) ensure there is a sql_config directory in phobos-lab directory
# -- 2) make a environment variable named PHOBOSSQLCONFIGPATH
#    that points to location of sql_config dir
# -- 3) Ensure timescale.pem is located in users Home dir

# -- Currently only darwin and linux platforms are supported


import os
import tempfile
from configparser import ConfigParser
from dataclasses import dataclass, field
from sys import platform
from typing import Any, Dict, Iterator, List, Optional, Tuple, TypeVar, Union

import asyncpg
import pandas as pd
import psycopg2
from paramiko import Ed25519Key
from psycopg2.extras import DictCursor, execute_values
from sshtunnel import SSHTunnelForwarder


@dataclass
class PlatformNotSupportedError(Exception):
    pass


@dataclass
class SectionNotExists(Exception):
    pass

path_to_secret_key = os.path.expanduser("~/timescale.pem")


# custom Type to represent psycopg2 connection and sshtunnel
connection = TypeVar("connection")
sshtunnel = TypeVar("sshtunnel")

CONFIG_PATH = os.getenv("PHOBOSSQLCONFIGPATH")
CONFIG_FILE = os.path.join(CONFIG_PATH, "config.ini")


@dataclass
class DBTester:
    section: str = field(init=False, default="phobosquantdev-dev")
    tunnel: Optional[sshtunnel] = field(init=False, default=None)
    pkey: Optional[Ed25519Key] = field(init=False, default=None)
    column_names: List[str] = field(init=False, default_factory=list)

    def __post_init__(self) -> None:
        self.pkey = Ed25519Key.from_private_key_file(path_to_secret_key)
        self.tunnel = self._create_tunnel()
        self.tunnel.start()

    async def __aenter__(self):
        self.tunnel = self._create_tunnel()
        self.tunnel.start()
        self.conn = await self.async_connect()
        return self

    async def __aexit__(self, *args):
        await self.conn.close()
        self.tunnel.close()

    def _create_tunnel(self) -> sshtunnel:
        tunnel = SSHTunnelForwarder(
            ("204.236.250.15", 22),
            ssh_username="ubuntu",
            ssh_pkey=self.pkey,
            remote_bind_address=("localhost", 5432),
        )
        return tunnel

    def _read_db_config(self) -> Dict[str, str]:
        """reads database configuration from config.ini file

        Returns
        -------
        Dict
            contains database configuration

        Raises
        ------
        Exception
            raises error if wrong environment is chosen
        """
        if not os.path.exists(CONFIG_PATH):
            raise FileNotFoundError(f"config_path: {CONFIG_FILE}")

        parser = ConfigParser()
        parser.read(CONFIG_FILE)

        # get the section, default to postgressql
        config = {}
        section = self.section
        if parser.has_section(section):
            params = parser.items(section)
            for param in params:
                config[param[0]] = param[1]
        else:
            raise SectionNotExists(f"Incorrect Section {section} provided")
        return config

    async def async_connect(self) -> Optional[connection]:
        """Connects to the postgresql securities_master db

        Returns
        -------
        Optional[connection]
            a connection object or None if failed to connect
        """
        try:
            if self.tunnel.is_active:
                params = self._read_db_config()
                port = self.tunnel.local_bind_port
                conn = await asyncpg.connect(**params, port=port)
                return conn
        except asyncpg.PostgresConnectionError() as error:
            print(error)

    def connect(self) -> Optional[connection]:
        """Connects to the postgresql securities_master db

        Returns
        -------
        Optional[connection]
            a connection object or None if failed to connect
        """
        try:
            if self.tunnel.is_active:
                port = self.tunnel.local_bind_port
            params = self._read_db_config()
            conn = psycopg2.connect(**params, port=port)
            return conn
        except psycopg2.DatabaseError as error:
            print(error)

    async def async_fetch(self, query: str):
        rows = await self.conn.fetch(query)
        return rows

    def fetch(self, query: str) -> Optional[Tuple[Dict[str, Any]]]:
        """Returns data associated with the table

        Parameters
        ----------
        query : str
            [description]

        Returns
        -------
        Optional[Tuple[Dict[str, Any]]]
            Tuple of Table rows
        """

        try:
            conn = self.connect()
            if conn:
                with conn.cursor(cursor_factory=DictCursor) as curr:
                    curr.execute(query)
                    self.column_names = [col.name for col in curr.description]
                    rows = curr.fetchall()
                conn.close()
                return rows
        except psycopg2.DatabaseError as error:
            print(error)

    def fetchdf(self, query: str) -> pd.DataFrame:
        """Returns a pandas dataframe of the db query"""
        return pd.DataFrame(self.fetch(query), columns=self.column_names)

    def drop(self, table_name: str) -> None:
        """removes table given by table_name from dev db

        Parameters
        ----------
        table_name : str
            the table in database
        """
        self.execute(f"drop table {table_name};")

    def execute(self, query: Union[str, Tuple[str]]) -> None:
        """Executes an query statement

        Parameters
        ----------
        query : Union[str, Tuple[str]]
            a single statement or tuple of queries
        """
        try:
            conn = self.connect()
            if conn:
                with conn.cursor() as curr:
                    if isinstance(query, str):
                        curr.execute(query)
                    elif isinstance(query, tuple):
                        for q in query:
                            curr.execute(q)
                conn.commit()
                conn.close()
        except Exception as e:
            print(e)

    def push(
        self,
        data: Iterator[Tuple[str, ...]],
        table_name: str,
        columns: List[str],
    ) -> None:
        """Pushes data to postgresql database
        Parameters
        ----------
        data : Iterator[Dict[str, Any]]
            to push to postgresql database
        table_name : str
            the table to push to
        columns : List[str]
            column names of the data
        Raises
        ------
        psycopg2.DatabaseError
            if the table doesn't exist or incorrect data format
        """
        conn = self.connect()
        cursor = conn.cursor()
        # get the column names
        col_names = ",".join(columns)
        query = f"""INSERT INTO {table_name} ({col_names}) values %s"""
        # insert data into database and close connection
        try:
            execute_values(cursor, query, data)
            conn.commit()
            cursor.close()
        except (Exception, psycopg2.DatabaseError) as e:
            print("error: ", e)
            conn.rollback()
            cursor.close()
        finally:
            conn.close()

    def copy_from_csv(
        self, data: pd.DataFrame, table_name: str, name: Optional[str] = ""
    ) -> None:
        """Copies data to table_name in securities_master db

        Parameters
        ----------
        data : pd.DataFrame
            large data to be pushed to table_name
        table_name : str
            table in securities_master
        name : Optional[str],
            name of the data, default=''
        """
        if data is None:
            return
        size = data.shape[0]
        print(f"Ready to push {('' if not name else name)} with size {size} rows")
        with tempfile.TemporaryDirectory() as tmp_dir:
            out_path = os.path.join(tmp_dir, "output.csv")
            data.to_csv(out_path, index=False, header=False)
            with open(out_path, "r") as f:
                next(f)
                conn = self.connect()
                cursor = conn.cursor()
                try:
                    cursor.copy_from(f, table_name, sep=",", null="")
                    conn.commit()
                    cursor.close()
                except (Exception, psycopg2.DatabaseError) as e:
                    print("error: ", e)
                    conn.rollback()
                    cursor.close()
                finally:
                    conn.close()

        print(f"pushed {('' if not name else name)} to {table_name} table")



db = DBTester()
print(db.fetchdf("select * from min_bars limit 10"))


         time exchange fsym tsym  close   high    low   open  volume_from  \
0  1637798460   CCCAGG  LPT  USD  46.36  46.36  46.18  46.18       542.92   
1  1637798520   CCCAGG  LPT  USD  46.42  46.45  46.34  46.36       955.49   
2  1637798580   CCCAGG  LPT  USD  46.45  46.45  46.29  46.42       166.54   
3  1637798640   CCCAGG  LPT  USD  46.40  46.46  46.38  46.45       150.03   
4  1637798700   CCCAGG  LPT  USD  46.32  46.40  46.27  46.40        91.51   
5  1637798760   CCCAGG  LPT  USD  46.26  46.32  46.26  46.32        32.74   
6  1637798820   CCCAGG  LPT  USD  46.33  46.37  46.26  46.26        91.84   
7  1637798880   CCCAGG  LPT  USD  46.19  46.38  46.19  46.33       105.44   
8  1637798940   CCCAGG  LPT  USD  46.14  46.20  46.14  46.19        98.51   
9  1637799000   CCCAGG  LPT  USD  46.14  46.18  46.14  46.14       243.46   

   volume_to  
0   25145.55  
1   44366.08  
2    7722.59  
3    6969.37  
4    4239.16  
5    1515.22  
6    4255.69  
7    4882.38  
8    4548.48  
9 

In [1]:
from kraken import KrakenBlotter, KrakenOrderBook

ModuleNotFoundError: No module named 'kraken'